In [1]:
import json
from returns.result import safe
from returns.future import future_safe, Future
from returns.pointfree import bind_result
from returns.pipeline import pipe
from returns.converters import  flatten
from pathlib import Path
import aiofiles
from typing import Any
from gnn.utils import read_file, parse_json

In [3]:
ROOT = Path(".");

In [5]:
from dotenv import load_dotenv
import os
from gnn.json_graph_loader import AsyncJSONToNeo4j, process_directory

In [6]:
load_dotenv();
data_directory = os.getenv("DATA_DIRECTORY");

In [8]:
async with AsyncJSONToNeo4j(database="neo4j") as jn4j:
    print(await process_directory(Path(data_directory), jn4j.process_json))
            # Step 2: Project into GDS
    print(await jn4j.project_graph_in_gds())

        # Step 3: (Optional) Estimate memory usage for Leiden
    print(await jn4j.estimate_leiden())

        # Step 4: Run Leiden in write mode
        # Write the final community ID back to Neo4j under 'communityId'
        # with concurrency = 4, no weight property, no intermediate communities
    result = await jn4j.run_leiden_write(
            write_property="communityId",
            concurrency=4,
            relationship_weight_property=None,
            include_intermediate=False
        )
    print("Leiden write result:", result)

2025-01-18 18:23:00.911 | INFO     | gnn.json_graph_loader:__aenter__:28 - Establishing connection to Neo4j
2025-01-18 18:23:00.914 | INFO     | gnn.json_graph_loader:process_directory:317 - Processing directory: data
2025-01-18 18:23:00.915 | INFO     | gnn.json_graph_loader:process_file:308 - Processing file: data/20230811_TurningMachineLayout_2min_StatisticsModule-1-3-29.json
2025-01-18 18:23:00.918 | DEBUG    | gnn.json_graph_loader:create_node:56 - Creating node with label: Root, properties: {'name': 'root'}
2025-01-18 18:23:00.931 | DEBUG    | gnn.json_graph_loader:create_node:56 - Creating node with label: Author, properties: {'name': 'Author', 'value': 'DUALIS'}
2025-01-18 18:23:00.933 | DEBUG    | gnn.json_graph_loader:create_relationship:77 - Creating relationship: Root -> Author [HAS]
2025-01-18 18:23:00.935 | DEBUG    | gnn.json_graph_loader:create_node:56 - Creating node with label: Date, properties: {'name': 'Date', 'value': '2023-08-11'}
2025-01-18 18:23:00.936 | DEBUG  

[<IOResult: <Success: <IOResult: <Success: None>>>>]
<IOResult: <Success: None>>
<IOResult: <Success: <Record nodeCount=291 relationshipCount=1134 requiredMemory='[608 KiB ... 635 KiB]'>>>
Leiden write result: <IOResult: <Failure: 'modularityGain'>>
